## This script contains
## 1. Install libraries and datasets
## 2. Input Section
## 3. Analysis Section

### 1. Install libraries and datasets

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "C:\\Users\\jwag1\\NBA\\player_game_logs_2020_2021_to_current.csv"
df = pd.read_csv(path)

In [3]:
# View first 5 rows of data
df.head()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,PlayerName
0,22020,1630173,22001069,"MAY 16, 2021",MIA @ DET,W,42,10,16,0.625,...,10,0,2,1,2,3,23,6,1,Precious Achiuwa
1,22020,1630173,22001062,"MAY 15, 2021",MIA @ MIL,L,2,1,2,0.500,...,0,1,0,0,0,0,2,3,1,Precious Achiuwa
2,22020,1630173,22001050,"MAY 13, 2021",MIA vs. PHI,W,2,0,0,0.000,...,0,0,0,0,0,0,0,-5,1,Precious Achiuwa
3,22020,1630173,22000986,"MAY 04, 2021",MIA vs. DAL,L,6,1,2,0.500,...,1,1,0,1,0,0,2,12,1,Precious Achiuwa
4,22020,1630173,22000969,"MAY 02, 2021",MIA @ CHA,W,1,0,1,0.000,...,0,0,0,0,0,0,0,-3,1,Precious Achiuwa


In [4]:
# View columns
print(df.columns)

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'PlayerName'],
      dtype='object')


### 2. Input Section

#### Criteria and Columns to Summarize
Update the criteria dictionary and columns to summarize list below:

In [7]:
# Example criteria
criteria = {
    'REB': {'min': 9.4, 'max': 10.4},
    'MIN': {'min': 34, 'max': 35}
     # Example criterion, adjust as needed
}

# Columns to summarize
columns_to_summarize = ['REB', 'MIN']

### 3. Analysis Section

Run the analysis using the criteria and columns specified above:

In [8]:
# Define the columns available for criteria
available_columns = [
    'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 
    'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS'
]

def filter_game_logs(df, criteria):
    """Filter the DataFrame based on the provided criteria."""
    query = ' & '.join([f"({col} >= {criteria[col]['min']}) & ({col} <= {criteria[col]['max']})" for col in criteria])
    return df.query(query)

def calculate_summary(df, columns):
    """Calculate the count and percentage for each unique value in the specified columns."""
    summaries = {}
    for col in columns:
        counts = df[col].value_counts().sort_index()
        percentages = (counts / len(df)) * 100
        cumulative_counts = counts[::-1].cumsum()[::-1]
        cumulative_percentages = (cumulative_counts / len(df)) * 100

        summaries[col] = pd.DataFrame({
            col: counts.index,
            'Count': counts.values,
            'Percentage': percentages.values,
            'Cumulative Count (>=)': cumulative_counts.values,
            'Cumulative Percentage (>=)': cumulative_percentages.values
        })
    return summaries

# Function to run the analysis
def run_analysis(df, criteria, columns_to_summarize):
    # Group by player and season to find players who meet the criteria for each season
    grouped = df.groupby(['PlayerName', 'SEASON_ID']).agg({col: 'mean' for col in criteria.keys()}).reset_index()

    # Filter to include only players who meet the criteria in at least one season
    criteria_conditions = [(grouped[col] >= criteria[col]['min']) & (grouped[col] <= criteria[col]['max']) for col in criteria]
    players_meeting_criteria = grouped.loc[sum(criteria_conditions) == len(criteria)]

    # Extract game logs for players meeting the criteria in the specified seasons
    filtered_logs = pd.merge(df, players_meeting_criteria[['PlayerName', 'SEASON_ID']], on=['PlayerName', 'SEASON_ID'], how='inner')

    # Display the count of game logs that meet the criteria
    log_count = len(filtered_logs)
    print(f"Number of game logs that meet the criteria: {log_count}")

    # Display the filtered game logs
    print("Filtered Game Logs:")
    print(filtered_logs)

    # Get summaries
    summaries = calculate_summary(filtered_logs, columns_to_summarize)

    # Display the summaries
    for col, summary in summaries.items():
        print(f"\n{col} Summary:")
        print(summary)

    return filtered_logs, summaries

# Run the analysis
filtered_logs, summaries = run_analysis(df, criteria, columns_to_summarize)

Number of game logs that meet the criteria: 137
Filtered Game Logs:
     SEASON_ID  Player_ID   Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0        22023    1628389  22301189  APR 14, 2024  MIA vs. TOR  W   19    8   
1        22023    1628389  22301176  APR 12, 2024  MIA vs. TOR  W   25    8   
2        22023    1628389  22301161  APR 10, 2024  MIA vs. DAL  L   30    3   
3        22023    1628389  22301147  APR 09, 2024    MIA @ ATL  W   48    4   
4        22023    1628389  22301133  APR 07, 2024    MIA @ IND  L   37    8   
..         ...        ...       ...           ...          ... ..  ...  ...   
132      22022     203954  22200059  OCT 26, 2022    PHI @ TOR  L   38   12   
133      22022     203954  22200044  OCT 24, 2022  PHI vs. IND  W   28    8   
134      22022     203954  22200028  OCT 22, 2022  PHI vs. SAS  L   36   14   
135      22022     203954  22200015  OCT 20, 2022  PHI vs. MIL  L   36    6   
136      22022     203954  22200001  OCT 18, 2022    PHI @ BOS 